In [1]:
import os
import imageio
from IPython.display import display, Image
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip "drive/MyDrive/test.zip" -d "/content/data/"

In [ ]:
classes = ['yes','no','up','down','left','right','on','off','stop','go','silence','unknown']
num_classes = 12

In [ ]:
model = models.load_model("drive/MyDrive/best_mfcc.h5")
#model.summary()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,loss="categorical_crossentropy",metrics=['accuracy'])

In [ ]:
image_shape = (299,299,3)
test_path = '/content/data/test/mfcc/'

def read_images(num=1000,image_names=[]):
  images = []
  names = []
  for k in range(min(num,len(image_names))):
    if (image_names[k][0] == "."):
      continue
    if (not os.path.exists(test_path + image_names[k])):
      continue 
    names.append(image_names[k])
    x = np.array(imageio.imread(test_path + image_names[k]))
    x = np.expand_dims(x,axis=0)
    x = np.resize(x,image_shape)
    images.append(x)
  return (names,np.array(images))

In [ ]:
def predict_labels(test_x):
  predictions = model.predict(test_x)
  labels = []
  for p in predictions:
    i = np.argmax(p)
    labels.append(classes[i])
  return labels 

In [ ]:
images = os.listdir(test_path)
test_num = len(images)
batch_size = 2000

In [ ]:
pred_file = open("predictions.csv","w")
pred_file.write("fname,label\n")
for n in range(0,test_num,batch_size):
  image_names, image_batch = read_images(batch_size,images)
  print(n)
  label_batch = predict_labels(image_batch)
  for k in range(min(batch_size,len(image_batch))):
    label = label_batch[k]
    im = image_names[k].replace("png","wav")
    pred_file.write("{},{}\n".format(im,label))
  
  images = images[len(image_batch):]
  if (len(images) == 0):
    break

pred_file.close()